# MVC 三层架构

- 贫血模型：只包含数据，不包含业务逻辑的类，就叫作贫血模型。贫血模型将数据与操作分离，破坏了面向对象的封装特性，是一种典型的面向过程的编程风格
- 充血模型：数据和对应的业务逻辑被封装到同一个类中，是典型的面向对象编程风格
- 领域驱动设计：即 DDD，主要是用来指导如何解耦业务系统，划分业务模块，定义业务领域模型及其交互。

基于充血模型的 DDD 开发模式实现的代码，也是按照 MVC 三层架构分层的。Controller 层还是负责暴露接口，Repository 层还是负责数据存取，Service 层负责核心业务逻辑。它跟基于贫血模型的传统开发模式的区别主要在 Service 层。基于贫血模型的传统的开发模式，比较适合业务比较简单的系统开发。相对应的，基于充血模型的 DDD 开发模式，更适合业务复杂的系统开发。

[VO，BO，PO，DO，DTO 的区别](https://zhuanlan.zhihu.com/p/102389552)

## 为什么贫血模型盛行

下面几项自己都中过招（环境问题和个人问题）：

### 环境问题

- 近朱者赤，近墨者黑
  - 大多数人都是模仿别人的代码，而别人的代码基本上都是 demo，没有复杂的业务逻辑，基本是贫血模型
  - 找不到好的指导与学习对象
- 接触不到复杂业务项目
  - 做 web 项目的，很大一部分就是简单的 CURD，贫血模型就能解决
- 公司以任务数来衡量个人价值

### 个人问题

- 不考虑项目质量属性
  - 只关心当前业务，没有意识去思考后期该如何维护和响应业务变更
- 求快不求质
  - 个人以任务数来自我满足
  - 没有 60 分和 100 分的概念
  - 需求分析、设计、编码合为一体

## 如何理解充血模型

先推荐一本书：整洁架构设计

先说一下充血模型中各组件的角色：

- controller 主要服务于非业务功能，比如说数据验证
- service 服务于 use case，负责的是业务流程与对应规则
- Domain 服务于核心业务逻辑和核心业务数据
- rep 用于与外部交互数据

---

额外说一点，业务开发个人倾向于六边形架构，而非传统的三层架构。六边形架构更能体现当下 web 应用的场景

六边形项目结构（根据实际情况自行组织与定义）：

- InboundHandler 代替 controller
  - \*WebController：处理 web 接口
  - \*WeChatController：处理微信公众号接口
  - \*AppController：处理 app 接口
  - \*MqListener：处理 消息
  - \*RpcController：处理子系统间的调用
- service 服务于 use case，负责的是业务流程与对应规则
  - CQPS + SRP：读写分离和单一原则将 use case 分散到不同的 service 中，避免一个巨大的 service 类（碰到过 8000 行的 service）
- Domain 服务于核心业务逻辑和核心业务数据
  - 最高层组件，不会依赖底层组件
  - 易测试
- outBoundhandle 代替 rep
  - MqProducer：发布消息
  - Cache：从缓存获取数据
  - sql：从数据库获取数据
  - Rpc：从子系统获取数据

---

各层之间的数据模型不要共用，主要是因为稳定性不同，各层数据模型的变更原因和变更速率是不同的，离 IO 设备越近的的稳定性越差，比如说 controller 层的 VO，rep 层的 entity。Domain 层是核心业务逻辑和核心业务数据，稳定性是最高的

---

几个不太容易理解的点（我刚开始碰到的时候很费解）：

- use case 和 核心业务逻辑该如何定义与区分
  - 哪些该放到 service 里面，哪些该放到 Domain 中
- rep 是依赖于 service 的，而不是 service 依赖 rep 层
  - 业务逻辑是最高层组件（最稳定的），rep 层是底层组件
- 接口能反转依赖关系

---

一剂良药：所有的中间层都是为了解耦


真正的业务逻辑都放在充血的领域对象中，与具体使用什么框架（比如 Spring，MyBatis），具体使用什么数据库无关。这样有利于保护领域对象中的数据，比如钱包中的余额，当有入账和出账操作时，余额在领域对象中自动执行加减操作，而不是将余额暴露在 Service 中直接操作（这样很容易出错可能导致帐不平衡，余额应该封装保护起来），当然“余额自动增减”这只是一个简单的业务逻辑例子，业务逻辑越复杂就越应该封装到领域对象中。

1. Service 层只是一个中间层，起到连接和组合作用。
   用于支持领域模型层和 Repository 层的交互（连接作用），利用各种领域对象执行业务逻辑（组合作用）。
   比如通过 Repository 查出数据，将数据转换为领域模型对象，利用领域模型对象执行业务逻辑（核心），然后调用 Repository 更新领域模型中的数据。

2. Service 类还负责一些非功能性及与三方系统交互的工作。
   比如幂等、事务、发邮件、发消息、记录日志、调用其他系统的 RPC 接口等。

不允许 Service 中的逻辑过于复杂，如果 Service 中的组合的业务逻辑过于复杂，我们就要将这业务逻辑抽取出一个新的领域对象进行封装，通过调用这个领域对象来进行这些复杂的操作。

由于 controller 和 Repository 层中本身没有什么业务逻辑，controller 中的 Vo 对象实际上只是传输数据使用（数据从系统传输数据到外部调用方），Repository 中的 Entity 本质上也只是传输数据（数据从数据库中传输数据到系统），所以用贫血模型不会带来副作用，是没有问题的。

针对框架、类库、组件等非业务系统的开发，其中一个比较大的难点就是，需求一般都比较抽象、模糊，需要你自己去挖掘，做合理取舍、权衡、假设，把抽象的问题具象化，最终产生清晰的、可落地的需求定义。需求定义是否清晰、合理，直接影响了后续的设计、编码实现是否顺畅。所以，作为程序员，你一定不要只关心设计与实现，前期的需求分析同等重要。

需求分析的过程实际上是一个不断迭代优化的过程。我们不要试图一下就能给出一个完美的解决方案，而是先给出一个粗糙的、基础的方案，有一个迭代的基础，然后再慢慢优化，这样一个思考过程能让我们摆脱无从下手的窘境。

一句话：使用进化算法的思想，提出一个 MVP（最小可行性产品），逐步迭代改进。

拿到这个需求，假设我们不了解接口鉴权，需求又不明确，我会我自己如下问题：

1. 什么叫接口鉴权？搞清基本概念
2. 接口鉴权最佳实践是什么？技术调研
3. appid 和 secret key 从哪里来？用户自己申请还是我们授权？用户申请是以什么方式申请（网页还是邮件？申请的网页有人做了么？）追问下去。
4. appid secretkey 存储在什么地方呢？数据存储
5. 用户如何使用？需要为用户提供接口鉴权使用手册和文档，及示例代码。写用户手册，文档。
6. 这个功能如何测试？提前想好如何测试
7. 接口鉴权功能何时上线？估计工期
8. 鉴权成功或失败返回码和信息定义？约定返回结果

关于防止重放攻击：请求参数中还可以加入 nonce（随机正整数），两次请求的 nonce 不能重复，timestamp 和 nonce 结合进一步防止重放攻击。
